In [1]:
!pip install sentencepiece

import pandas as pd 
import sentencepiece as spm 
import urllib.request 
import csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.5 MB/s 


In [2]:
!pip install konlpy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 331 kB/s 
     |████████████████████████████████| 465 kB 51.2 MB/s 


In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt") 

naver_df = pd.read_table('ratings.txt')

naver_df = naver_df.dropna(how = 'any') # Null 값이 존재하는 행 제거 

# 결과를 naver_review.txt 파일에 저장
with open('naver_review.txt', 'w', encoding='utf8') as f:
	f.write('\n'.join(naver_df['document']))

In [4]:
 
train_data = pd.read_table('naver_review.txt', header = None)
#print(train_data)
print("문장수  : " ,len(train_data))


문장수  :  198782


In [5]:
train_data[0].nunique()

193390

In [6]:
# document 열의 중복 제거
train_data.drop_duplicates(subset=[0], inplace=True)
print('총 샘플의 수 :',len(train_data))

총 샘플의 수 : 193390


In [7]:
# train_data[0] = train_data[0].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# train_data[:5]

In [8]:
train_data[0] = train_data[0].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data[0].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())
print(train_data.isnull())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


0    0
dtype: int64
            0
0       False
1       False
2       False
3       False
4       False
...       ...
198777  False
198778  False
198779  False
198780  False
198781  False

[193390 rows x 1 columns]


In [9]:
train_data = train_data.dropna(how = 'any')
print(train_data)
print("총 문장수 : ",len(train_data))

                                                        0
0                                     어릴때보고 지금다시봐도 재밌어요ㅋㅋ
1       디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...
2                    폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.
3       와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...
4                             안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.
...                                                   ...
198777                                     포켓 몬스터 짜가 ㅡㅡ;;
198778                                              쓰.레.기
198779                  완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.
198780                왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ
198781                                    포풍저그가나가신다영차영차영차

[193390 rows x 1 columns]
총 문장수 :  193390


In [10]:
count = 0
for i in train_data[0]:
  spword = i.split()

  count += len(spword)
print("총 단어수  : " ,count)

  #print(i)
# w = train_data[0]
# print(w)
# lines = w[90001:90011].tolist()
#print(lines)

총 단어수  :  1509809


In [11]:
# stopwords = ['의','가','이','을','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [12]:
# print(X_train[:3])

In [13]:
X_train = []
okt = Okt()
for sentence in tqdm(train_data[0]):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    X_train.append(tokenized_sentence)

100%|██████████| 193390/193390 [09:56<00:00, 324.35it/s]


In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [15]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('고유단어수  :',total_cnt)
#print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
#print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
#print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

고유단어수  : 56723
